In [ ]:
!pip install torch_geometric

In [2]:
import torch
from torch_geometric.nn import SAGEConv

In [22]:
class GraphSAGE(torch.nn.Module):
  def __init__(self, in_channels, hidden_channels, out_channels):
    super(GraphSAGE, self).__init__()
    self.conv1 = SAGEConv(in_channels, hidden_channels)
    self.conv2 = SAGEConv(hidden_channels, out_channels)

  def forward(self, x, edge_index):
    x = self.conv1(x, edge_index)
    x = torch.relu(x)
    x = self.conv2(x, edge_index)
    return x

In [23]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='/tmp/Cora', name = 'Cora')
data = dataset[0]

In [24]:
x = dataset.num_node_features
y = dataset.num_classes

model = GraphSAGE(x, 16, y)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [25]:
model

GraphSAGE(
  (conv1): SAGEConv(1433, 16, aggr=mean)
  (conv2): SAGEConv(16, 7, aggr=mean)
)

In [14]:
model.train()
for epoch in range(200):
  optimizer.zero_grad()
  out = model(data.x, data.edge_index)
  loss = torch.nn.functional.cross_entropy(out[data.train_mask], data.y[data.train_mask])
  loss.backward()
  optimizer.step()

In [15]:
model.eval()
_, pred = model(data.x, data.edge_index).max(dim = 1)
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7340


Using Mini-Batches

In [28]:
from torch_geometric.data import DataLoader

loader = DataLoader(dataset, batch_size = 32, shuffle = True)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [32]:
for epoch in range(501):
  for batch in loader:
    optimizer.zero_grad()
    x, edge_index = batch.x, batch.edge_index
    out = model(x, edge_index)
    loss = torch.nn.functional.cross_entropy(out, batch.y)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
      print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 000, Loss: 0.0003
Epoch: 100, Loss: 0.0002
Epoch: 200, Loss: 0.0001
Epoch: 300, Loss: 0.0001
Epoch: 400, Loss: 0.0001
Epoch: 500, Loss: 0.0001
